## Tensor定义
tensor张量可以理解为n维数组:
* 0维张量是一个数(Scalar/number),
* 1维张量是向量(Vector),
* 2维张量是矩阵(Martrix),
* 以此类推...

## 基础运算

In [2]:
import tensorflow as tf
a=tf.add(3,5)
print(a)

Tensor("Add:0", shape=(), dtype=int32)


TF的加法方法，但是通常的赋值并不是正常运行加法。
需要将被赋值的变量a放入session运行才能看到运算结果。

In [4]:
a=tf.add(3,5)
sess=tf.Session()
print(sess.run(a))
sess.close()

8


将运算结果存入sess稍后再用的写法

In [6]:
a=tf.add(3,5)
with tf.Session() as sess:
    print(sess.run(a))

8


tf.Session()封装了一个执行运算的环境，用tensor对象内的赋值进行运算

## 混合运算

In [11]:
x=2
y=3
op1 =tf.add(x,y)
op2=tf.multiply(x,y)
op3=tf.pow(op2,op1)
with tf.Session() as sess:
    op3=sess.run(op3)
    print(op3)

7776


## Subgraphs

In [13]:
x=2
y=3
add_op=tf.add(x,y)
mul_op=tf.multiply(x,y)
useless=tf.multiply(x,add_op)
pow_op=tf.pow(add_op,mul_op)
with tf.Session() as sess:
    z=sess.run(pow_op)
    print(z)

15625


由于求Z值并不需要计算useless部分，所以session并没有计算它

In [15]:
x=2
y=3
add_op=tf.add(x,y)
mul_op=tf.multiply(x,y)
useless=tf.multiply(x,add_op)
pow_op=tf.pow(add_op,mul_op)
with tf.Session() as sess:
    z,not_useless=sess.run([pow_op,useless])
    print(z)
    print(not_useless)

15625
10


同时进行两个计算

## Graph

In [20]:
g=tf.Graph()
with g.as_default():
    x=tf.add(3,5)
    
sess=tf.Session(graph=g)
with tf.Session() as sess: #此处两行的打包方式已经过时，如果报错需要改成下面的格式
    sess.run(g)


TypeError: Fetch argument <tensorflow.python.framework.ops.Graph object at 0x0000018E70371A20> has invalid type <class 'tensorflow.python.framework.ops.Graph'>, must be a string or Tensor. (Can not convert a Graph into a Tensor or Operation.)

教程的例子有报错，需要改成下面的格式

In [24]:
g=tf.Graph()
with g.as_default():
    x=tf.add(3,5)
    

with tf.Session(graph=g) as sess: #将上面的两行改成一行
    sess.run(x)                   #不能直接运行graph


g=tf.Graph()
with g.as_default():
    a=3
    b=5
    x=tf.add(a,b)
sess = tf.Session(graph=g)
sess.close()

向graph内添加加法运算，并且设为默认graph

In [29]:
g1=tf.get_default_graph()
g2=tf.graph()
#将运算加入到默认graph
with g1.as_default():
    a=tf.Constant(3)       #不会报错，但推荐添加到自己创建的graph里
    
#将运算加入到用户创建的graph
with g2.as_default():
    b=tf.Constant(5)

建议不要修改默认graph

** Graph 的优点 **

* 节省运算资源，只计算需要的部分
* 将计算分解为更小的部分
* 让分布式运算更方便，向多个CPU，GPU或其它设备分配任务
* 适合那些使用directed graph的机器学习算法

### Graph 与 Session 的区别

* Graph定义运算，但不计算任何东西，不保存任何数值，只存储你在各个节点定义的运算。
* Session可运行Graph或一部分Graph，它负责在一台或多台机器上分配资源，保存实际数值，中间结果和变量。


下面通过以下例子具体阐明二者的区别：

In [ ]:
graph=tf.Graph()
with graph.as_default():#每次TF都会生产默认graph,所以前两行其实并不需要
    variable=tf.Variable(42,name='foo')
    initialize=tf.global_variables_initializer()
    assign=variable.assign(13)

创建变量，初始化值42，之后赋值13

In [22]:
graph=tf.Graph()
with graph.as_default():#每次TF都会生产默认graph,所以前两行其实并不需要
    variable=tf.Variable(42,name='foo')
    initialize=tf.global_variables_initializer()
    assign=variable.assign(13)

with tf.Session(graph=graph) as sess:  
    sess.run(initialize)      #记得将计算步骤在此处列出来
    sess.run(assign)
    print(sess.run(variable))

13


定义的计算数量达到三个时就要使用graph。但是variable每次运算都要在session内run一遍，如果跳过此步骤，就无法获取运算后变量数值。（也就相当于没计算过）

In [25]:
graph=tf.Graph()
with graph.as_default():#每次TF都会生产默认graph,所以前两行其实并不需要
    variable=tf.Variable(42,name='foo')
    initialize=tf.global_variables_initializer()
    assign=variable.assign(13)

with tf.Session(graph=graph) as sess:
    print(sess.run(variable))    #未列出计算步骤所以报错

FailedPreconditionError: Attempting to use uninitialized value foo
	 [[Node: _retval_foo_0_0 = _Retval[T=DT_INT32, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](foo)]]

In [26]:
graph=tf.Graph()
with graph.as_default():#每次TF都会生产默认graph,所以前两行其实并不需要
    variable=tf.Variable(42,name='foo')
    initialize=tf.global_variables_initializer()
    assign=variable.assign(13)

with tf.Session(graph=graph) as sess:  
    sess.run(initialize)      #计算步骤，列到第几步就计算到第几步
    print(sess.run(variable))

42
